In [177]:
from abc import ABC, abstractmethod

class Reality():


    def __init__(
            self, person_list=[], broker_list=[], instrument_list=[], starting_time_period=0):
        self.person_list = person_list
        self.broker_list = broker_list
        self.instrument_list = instrument_list
        self.person_data = {}
        self.instrument_data = {}
        self.t = starting_time_period

    def add_person(self, person):
        if person in self.person_list:
            print('person already included')
            return
        self.person_list.append(person)

    def add_instrument(self, instrument):
        if instrument in self.instrument_list:
            print('instrument already included')
            return
        self.instrument_list.append(instrument)

    def execute_person_behaviour(self, person, behaviour_dict,
                                  income_change=0, acc_list=[], acc_instrument_dict={}):
        """Investing behaviour of person is always executed at the beginning of the period
        """

        person.change_monthly_income(income_change=income_change)
        person.receive_monthly_income()
        for acc in acc_list:
            person.add_account(booker_account=acc)
        for acc in acc_instrument_dict:
            for instrument_name in acc_instrument_dict[acc]:
                person.add_instrument_to_account(
                    account_name=acc, instrument_name=instrument_name)
        person.execute_investing_behaviour(behaviour_dict=behaviour_dict)
        self.save_person_data(person=person)

    def calculate_accs_totals(self, person):
        """Account totals of the person are calculated at the end of the period
        """

        for account in person.broker_accs:
            account.calculate_inst_amount_wraper()
        self.save_person_data(person)

    def calculate_all_peeople_accs_totals(self):
        for person in self.person_list:
            self.calculate_accs_totals(person=person)

    def execute_instruments_period(self):
        for instrument in self.instrument_list:
            self.execute_instrument_period(instrument=instrument)

    def execute_instrument_period(self, instrument):
        instrument.execute_time_period(t=self.t)
        self.save_instrument_data(instrument=instrument)

    def execute_people_behaviour(self, behaviour_dict):
        for person in self.person_list:
            self.execute_person_behaviour(person=person,
                                           behaviour_dict=behaviour_dict[person.name])

    def save_person_data(self, person):
        if person.name not in self.person_data:
              self.person_data[person.name] = {}
        self.person_data[person.name][self.t] = {}
        self.person_data[person.name][self.t]['amount'] = person.money
        self.person_data[person.name][self.t]['income'] = person.current_monthly_income
        self.person_data[person.name][self.t]['age'] = person.age
        self.person_data[person.name][self.t]['accounts'] = {}
        for account in person.broker_accs:
            self.person_data[person.name][self.t]['accounts'][account.broker_name] = {}
            self.person_data[person.name][self.t]['accounts'][account.broker_name]['amount'] = account.account_amount
            self.person_data[person.name][self.t]['accounts'][account.broker_name]['instruments'] = account.instrument_dict
    
    def save_instrument_data(self, instrument):
        if instrument.name not in self.instrument_data:
            self.instrument_data[instrument.name] = instrument.price_ts
            print(self.instrument_data)
        self.instrument_data[instrument.name].append(instrument.current_price)

    def execute_period(self, behaviour_dict):
        """changes to the accounts/indexes employed and adding money to accounts are decided at the beginning of the period
        calculation of account balances then happens at the end of the period"""

        self.t += 1
        self.execute_people_behaviour(behaviour_dict=behaviour_dict)
        self.execute_instruments_period()
        self.calculate_all_peeople_accs_totals()


class Person():

    def __init__(
            self, name, starting_age, starting_money, current_monthly_income, reality, broker_acounts=[]):
        self.name = name
        self.age = starting_age
        self.money = starting_money
        self.current_monthly_income = current_monthly_income 
        self.broker_accs = broker_acounts
        self.investing_behaviour = {}
        self.reality = reality

    def add_account(self, broker_name):
        if broker_name not in self.reality.broker_list:
            print("Broker does not exist in this reality")
            return
        account = Account(owner=self, broker_name=broker_name)
        self.broker_accs.append(account)

    def delete_account(self, account):
        self.broker_accs.remove(account)

    def add_instrument_to_account(self, account_name, instrument_name):
        acc = self.get_account_by_name(account_name=account_name)
        if acc is None:
            print('Account does not belong to the person - add inst')
            return
        acc_index =  self.broker_accs.index(acc)
        print(acc_index) 
        self.broker_accs[acc_index].instrument_dict[instrument_name] = 0
        print(self.broker_accs[acc_index].instrument_dict)

    def add_money_to_account(self, account_name, money):
        acc = self.get_account_by_name(account_name=account_name)
        if acc is None:
            print('Account does not belong to this person- add money acc')
            return
        acc_index =  self.broker_accs.index(acc)
        self.broker_accs[acc_index].account_amount += money

    def take_money_from_account(self, account_name, money):
        acc = self.get_account_by_name(account_name=account_name)
        if acc is None:
            print('Account does not belong to this person-take money acc')
            return
        acc_index =  self.broker_accs.index(acc)
        self.broker_accs[acc_index].account_amount -= money
        self.money += money

    def add_money_to_instrument(self, account, instrument_name, money):
        account.transfer_money_to_instrument(
            instrument_name=instrument_name, money=money)
        
    def receive_monthly_income(self):
        self.money = self.money + self.current_monthly_income

    def change_monthly_income(self, income_change):
        self.current_monthly_income = (self.current_monthly_income  
                                       + income_change)
                
    def execute_investing_behaviour(self, behaviour_dict):
        for account_name in behaviour_dict:
            account_total = sum(behaviour_dict[account_name].values())
            self.add_money_to_account(account_name=account_name,
                                       money=account_total
                                       *self.current_monthly_income)
            acc = self.get_account_by_name(account_name=account_name)
            for instrument_name in behaviour_dict[account_name]:
                self.add_money_to_instrument(
                    account=acc, instrument_name=instrument_name, 
                    money=(self.current_monthly_income
                          * behaviour_dict[account_name][instrument_name]))
                
    def get_account_by_name(self, account_name):
        print(account_name)
        for account in  self.broker_accs:
            print(account.broker_name)
            if account.broker_name == account_name:
                print('yes')
                return account

        
class Account():

    def __init__(self, owner, broker_name, starting_instrument_dict={}, starting_amount=0):
        self.broker_name = broker_name
        self.owner = owner
        self.instrument_dict = starting_instrument_dict
        self.account_amount  = starting_amount

    def add_instrument(self, instrument):
        self.instrument_dict[instrument.name] = 0
    
    def delete_instrument(self, instrument):
        if self.instrument_dict[instrument.name] != 0:
            print('can delete account, there are still money in it')
            return
        del self.instrument_dict[instrument.name]

    def calculate_instrument_amount(self, instrument_name):
        if instrument_name not in self.instrument_dict:
            print('Instrument is not active in current account')
            return
        curr_price = self.owner.reality.instrument_data[instrument_name][-1]
        prev_price = self.owner.reality.instrument_data[instrument_name][-2]
        multip = (curr_price - prev_price) / (curr_price)
        self.instrument_dict[instrument_name] = self.instrument_dict[instrument_name] * multip

    def calculate_inst_amount_wraper(self):
        for instrument_name in self.instrument_dict:
            self.calculate_instrument_amount(instrument_name=instrument_name)

    def transfer_money_to_instrument(self, instrument_name, money):
        if money > self.account_amount:
            print('Not enough money for the transaction')
            return
        self.account_amount = self.account_amount - money
        self.instrument_dict[instrument_name] += money


class  Instrument(ABC):

    def __init__(self, name, type_):
        self.name = name
        self.type = type_
        self.multip = 1

    @abstractmethod
    def calculate_multip(self):
        pass

    @abstractmethod
    def execute_time_period(self):
        pass


class Index(Instrument):

    def __init__(
            self, name, reality, type_='index', price_ts=[], current_price=None):
        super().__init__(name, type_)
        self.current_price = current_price
        self.previous_price = price_ts[-1]
        print(self.previous_price)
        self.price_ts = price_ts
        self.reality = reality

    def calculate_current_price(self, t, mma_length):
        if self.current_price is not None:
            self.previous_price = self.current_price 
        self.current_price = self.ts_calc(mma_length)
        self.price_ts[t] = self.current_price
        print(self.current_price)
        print(self.current_price)

    def ts_calc(self, mma_length):
        if len(self.price_ts) < mma_length:
            return('We need at least ' 
                  + str(mma_length) 
                  + " previous observations")
        current_price = ((sum(self.price_ts[len(self.price_ts) - (mma_length): ]))  / (mma_length))
        return current_price
            
    def calculate_multip(self):
        (self.current_price - self.previous_price) // self.previous_price

    def execute_time_period(self, t):
        self.calculate_current_price(t=t, mma_length=20)
        self.calculate_multip()


class TransactionAccount(Instrument):

    def __init__(self, name, type_='trans_account'):
        super().__init__(type_, name)

    def execute_time_period(self):
        pass




        



 

In [142]:
import pandas as pd
sap500 = pd.read_csv('sap500.csv')
sap500 = list(sap500["Close"])
sap500.reverse()
sap500 = [float(x.replace(",", "")) for x in sap500]

In [156]:
for x in sap500:
    if '3' not in str(x):
        print(x)

4019.65
4019.81
4016.95
4016.22
4070.56
4017.77
4076.6
4119.21
4179.76
4111.08
4164.0
4117.86
4081.5
4090.46
4147.6
4090.41
4079.09
4045.64
4048.42
4002.87
4027.81
4124.51
4100.6
4105.02
4109.11
4108.94
4091.95
4146.22
4154.87
4154.52
4129.79
4055.99
4169.48
4167.87
4119.58
4090.75
4061.22
4119.17
4124.08
4109.9
4158.77
4198.05
4191.98
4145.58
4115.24
4151.28
4205.45
4205.52
4221.02
4267.52
4298.86
4425.84
4409.59
4455.59
4446.82
4411.59
4472.16
4510.04
4505.42
4522.79
4554.98
4565.72
4554.64
4567.46
4566.75
4588.96
4501.89
4518.44
4467.71
4464.05
4489.72
4405.71
4514.87
4507.66
4515.77
4465.48
4451.14
4457.49
4487.46
4461.9
4467.44
4505.1
4402.2
4274.51
4299.7
4288.05
4229.45
4258.19
4278.0
4224.16
4217.04
4247.68
4186.77
4166.82
4415.24
4411.55
4495.7
4502.88
4508.24
4514.02
4556.62
4554.89
4550.58
4567.8
4569.78
4567.18
4585.59


In [143]:
sap500

[3933.92,
 3963.51,
 3934.38,
 3990.56,
 4019.65,
 3995.32,
 3895.75,
 3852.36,
 3817.66,
 3821.62,
 3878.44,
 3822.39,
 3844.82,
 3829.25,
 3783.22,
 3849.28,
 3839.5,
 3824.14,
 3852.97,
 3808.1,
 3895.08,
 3892.09,
 3919.25,
 3969.61,
 3983.17,
 3999.09,
 3990.97,
 3928.86,
 3898.85,
 3972.61,
 4019.81,
 4016.95,
 4016.22,
 4060.43,
 4070.56,
 4017.77,
 4076.6,
 4119.21,
 4179.76,
 4136.48,
 4111.08,
 4164.0,
 4117.86,
 4081.5,
 4090.46,
 4137.29,
 4136.13,
 4147.6,
 4090.41,
 4079.09,
 3997.34,
 3991.05,
 4012.32,
 3970.04,
 3982.24,
 3970.15,
 3951.39,
 3981.35,
 4045.64,
 4048.42,
 3986.37,
 3992.01,
 3918.32,
 3861.59,
 3855.76,
 3919.29,
 3891.93,
 3960.28,
 3916.64,
 3951.57,
 4002.87,
 3936.97,
 3948.72,
 3970.99,
 3977.53,
 3971.27,
 4027.81,
 4050.83,
 4109.31,
 4124.51,
 4100.6,
 4090.38,
 4105.02,
 4109.11,
 4108.94,
 4091.95,
 4146.22,
 4137.64,
 4151.32,
 4154.87,
 4154.52,
 4129.79,
 4133.52,
 4137.04,
 4071.63,
 4055.99,
 4135.35,
 4169.48,
 4167.87,
 4119.58,
 4090.7

In [178]:
reality_1 = Reality()
reality_1.broker_list.append('kb')
investing_behaviour = {'Terka': {'kb': {'SAP500': 0.7}}}
index_sap500 = Index(name='SAP500', reality=reality_1, price_ts=sap500)
reality_1.add_instrument(index_sap500)
terka = Person(name='Terka', starting_age=26, starting_money=35000,
               current_monthly_income=25000, reality=reality_1)
terka.add_account('kb')
print(terka.broker_accs)
reality_1.add_person(person=terka)
terka.add_instrument_to_account('kb', instrument_name='SAP500')
print('naem')
print(terka.broker_accs[0].broker_name)
print('name')
for t in range(500):
    reality_1.execute_period(behaviour_dict=investing_behaviour)

4585.59
kb
kb
yes
0
{'SAP500': 0}
naem
kb
name
kb
kb
yes
kb
kb
yes
4524.3365
4524.3365
{'SAP500': [3933.92, 4524.3365, 3934.38, 3990.56, 4019.65, 3995.32, 3895.75, 3852.36, 3817.66, 3821.62, 3878.44, 3822.39, 3844.82, 3829.25, 3783.22, 3849.28, 3839.5, 3824.14, 3852.97, 3808.1, 3895.08, 3892.09, 3919.25, 3969.61, 3983.17, 3999.09, 3990.97, 3928.86, 3898.85, 3972.61, 4019.81, 4016.95, 4016.22, 4060.43, 4070.56, 4017.77, 4076.6, 4119.21, 4179.76, 4136.48, 4111.08, 4164.0, 4117.86, 4081.5, 4090.46, 4137.29, 4136.13, 4147.6, 4090.41, 4079.09, 3997.34, 3991.05, 4012.32, 3970.04, 3982.24, 3970.15, 3951.39, 3981.35, 4045.64, 4048.42, 3986.37, 3992.01, 3918.32, 3861.59, 3855.76, 3919.29, 3891.93, 3960.28, 3916.64, 3951.57, 4002.87, 3936.97, 3948.72, 3970.99, 3977.53, 3971.27, 4027.81, 4050.83, 4109.31, 4124.51, 4100.6, 4090.38, 4105.02, 4109.11, 4108.94, 4091.95, 4146.22, 4137.64, 4151.32, 4154.87, 4154.52, 4129.79, 4133.52, 4137.04, 4071.63, 4055.99, 4135.35, 4169.48, 4167.87, 4119.58, 4090.7

In [183]:
reality_1.person_data

{'Terka': {1: {'amount': 60000,
   'income': 25000,
   'age': 26,
   'accounts': {'kb': {'amount': 0.0, 'instruments': {'SAP500': 0.0}}}},
  2: {'amount': 85000,
   'income': 25000,
   'age': 26,
   'accounts': {'kb': {'amount': 0.0, 'instruments': {'SAP500': 0.0}}}},
  3: {'amount': 110000,
   'income': 25000,
   'age': 26,
   'accounts': {'kb': {'amount': 0.0, 'instruments': {'SAP500': 0.0}}}},
  4: {'amount': 135000,
   'income': 25000,
   'age': 26,
   'accounts': {'kb': {'amount': 0.0, 'instruments': {'SAP500': 0.0}}}},
  5: {'amount': 160000,
   'income': 25000,
   'age': 26,
   'accounts': {'kb': {'amount': 0.0, 'instruments': {'SAP500': 0.0}}}},
  6: {'amount': 185000,
   'income': 25000,
   'age': 26,
   'accounts': {'kb': {'amount': 0.0, 'instruments': {'SAP500': 0.0}}}},
  7: {'amount': 210000,
   'income': 25000,
   'age': 26,
   'accounts': {'kb': {'amount': 0.0, 'instruments': {'SAP500': 0.0}}}},
  8: {'amount': 235000,
   'income': 25000,
   'age': 26,
   'accounts': {'k

In [5]:
a = Index(name='aa')

In [13]:
reality = Reality()
person = Person(name='ja', starting_age=25, starting_money=0, reality=reality, current_monthly_income=1250)

In [14]:
reality.broker_list.append('kb')

In [16]:
person.reality.broker_list

['kb']

In [10]:
person.reality.broker_list

In [4]:
ind_1 = Index(name='SAP', type_='etf')
acc = Account()